In [2]:
from skimage import io,transform
import glob
import os
import tensorflow as tf
import numpy as np
import time

#data path
path='flower_photo/'
#place to save model
model_path='data/model.ckpt'

#resize pics
w=100
h=100
c=3


#load images
def read_img(path):
    cate=[path+x for x in os.listdir(path) if os.path.isdir(path+x)]
    imgs=[]
    labels=[]
    for idx,folder in enumerate(cate):
        for im in glob.glob(folder+'/*.jpg'):
            print('reading the images:%s'%(im))
            img=io.imread(im)
            img=transform.resize(img,(w,h))
            imgs.append(img)
            labels.append(idx)
    return np.asarray(imgs,np.float32),np.asarray(labels,np.int32)
data,label=read_img(path)


#reorder
num_example=data.shape[0]
arr=np.arange(num_example)
np.random.shuffle(arr)
data=data[arr]
label=label[arr]


#divide everything to testing data and training data
ratio=0.8
s=np.int(num_example*ratio)
x_train=data[:s]
y_train=label[:s]
x_val=data[s:]
y_val=label[s:]

#-----------------build network----------------------
#-
x=tf.placeholder(tf.float32,shape=[None,w,h,c],name='x')
y_=tf.placeholder(tf.int32,shape=[None,],name='y_')

def inference(input_tensor, train, regularizer):
    with tf.variable_scope('layer1-conv1'):
        conv1_weights = tf.get_variable("weight",[5,5,3,32],initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv1_biases = tf.get_variable("bias", [32], initializer=tf.constant_initializer(0.0))
        conv1 = tf.nn.conv2d(input_tensor, conv1_weights, strides=[1, 1, 1, 1], padding='SAME')
        relu1 = tf.nn.relu(tf.nn.bias_add(conv1, conv1_biases))

    with tf.name_scope("layer2-pool1"):
        pool1 = tf.nn.max_pool(relu1, ksize = [1,2,2,1],strides=[1,2,2,1],padding="VALID")

    with tf.variable_scope("layer3-conv2"):
        conv2_weights = tf.get_variable("weight",[5,5,32,64],initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv2_biases = tf.get_variable("bias", [64], initializer=tf.constant_initializer(0.0))
        conv2 = tf.nn.conv2d(pool1, conv2_weights, strides=[1, 1, 1, 1], padding='SAME')
        relu2 = tf.nn.relu(tf.nn.bias_add(conv2, conv2_biases))

    with tf.name_scope("layer4-pool2"):
        pool2 = tf.nn.max_pool(relu2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    with tf.variable_scope("layer5-conv3"):
        conv3_weights = tf.get_variable("weight",[3,3,64,128],initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv3_biases = tf.get_variable("bias", [128], initializer=tf.constant_initializer(0.0))
        conv3 = tf.nn.conv2d(pool2, conv3_weights, strides=[1, 1, 1, 1], padding='SAME')
        relu3 = tf.nn.relu(tf.nn.bias_add(conv3, conv3_biases))

    with tf.name_scope("layer6-pool3"):
        pool3 = tf.nn.max_pool(relu3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    with tf.variable_scope("layer7-conv4"):
        conv4_weights = tf.get_variable("weight",[3,3,128,128],initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv4_biases = tf.get_variable("bias", [128], initializer=tf.constant_initializer(0.0))
        conv4 = tf.nn.conv2d(pool3, conv4_weights, strides=[1, 1, 1, 1], padding='SAME')
        relu4 = tf.nn.relu(tf.nn.bias_add(conv4, conv4_biases))

    with tf.name_scope("layer8-pool4"):
        pool4 = tf.nn.max_pool(relu4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
        nodes = 6*6*128
        reshaped = tf.reshape(pool4,[-1,nodes])

    with tf.variable_scope('layer9-fc1'):
        fc1_weights = tf.get_variable("weight", [nodes, 1024],
                                      initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer != None: tf.add_to_collection('losses', regularizer(fc1_weights))
        fc1_biases = tf.get_variable("bias", [1024], initializer=tf.constant_initializer(0.1))

        fc1 = tf.nn.relu(tf.matmul(reshaped, fc1_weights) + fc1_biases)
        if train: fc1 = tf.nn.dropout(fc1, 0.5)

    with tf.variable_scope('layer10-fc2'):
        fc2_weights = tf.get_variable("weight", [1024, 512],
                                      initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer != None: tf.add_to_collection('losses', regularizer(fc2_weights))
        fc2_biases = tf.get_variable("bias", [512], initializer=tf.constant_initializer(0.1))

        fc2 = tf.nn.relu(tf.matmul(fc1, fc2_weights) + fc2_biases)
        if train: fc2 = tf.nn.dropout(fc2, 0.5)

    with tf.variable_scope('layer11-fc3'):
        fc3_weights = tf.get_variable("weight", [512, 5],
                                      initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer != None: tf.add_to_collection('losses', regularizer(fc3_weights))
        fc3_biases = tf.get_variable("bias", [5], initializer=tf.constant_initializer(0.1))
        logit = tf.matmul(fc2, fc3_weights) + fc3_biases

    return logit

#---------------------------network ending---------------------------
regularizer = tf.contrib.layers.l2_regularizer(0.0001)
logits = inference(x,False,regularizer)

#give names to files for testing
b = tf.constant(value=1,dtype=tf.float32)
logits_eval = tf.multiply(logits,b,name='logits_eval') 

loss=tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y_)
train_op=tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
correct_prediction = tf.equal(tf.cast(tf.argmax(logits,1),tf.int32), y_)    
acc= tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


#load data
def minibatches(inputs=None, targets=None, batch_size=None, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batch_size + 1, batch_size):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batch_size]
        else:
            excerpt = slice(start_idx, start_idx + batch_size)
        yield inputs[excerpt], targets[excerpt]


#train and validate

n_epoch=10                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
batch_size=64
saver=tf.train.Saver()
sess=tf.Session()  
sess.run(tf.global_variables_initializer())
for epoch in range(n_epoch):
    start_time = time.time()

    #training
    train_loss, train_acc, n_batch = 0, 0, 0
    for x_train_a, y_train_a in minibatches(x_train, y_train, batch_size, shuffle=True):
        _,err,ac=sess.run([train_op,loss,acc], feed_dict={x: x_train_a, y_: y_train_a})
        train_loss += err; train_acc += ac; n_batch += 1
    print("   train loss: %f" % (np.sum(train_loss)/ n_batch))
    print("   train acc: %f" % (np.sum(train_acc)/ n_batch))

    #validation
    val_loss, val_acc, n_batch = 0, 0, 0
    for x_val_a, y_val_a in minibatches(x_val, y_val, batch_size, shuffle=False):
        err, ac = sess.run([loss,acc], feed_dict={x: x_val_a, y_: y_val_a})
        val_loss += err; val_acc += ac; n_batch += 1
    print("   validation loss: %f" % (np.sum(val_loss)/ n_batch))
    print("   validation acc: %f" % (np.sum(val_acc)/ n_batch))
saver.save(sess,model_path)
sess.close()

reading the images:flower_photo/roses\10090824183_d02c613f10_m.jpg


C:\Users\lyl5380\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


reading the images:flower_photo/roses\102501987_3cdb8e5394_n.jpg
reading the images:flower_photo/roses\10503217854_e66a804309.jpg
reading the images:flower_photo/roses\10894627425_ec76bbc757_n.jpg
reading the images:flower_photo/roses\110472418_87b6a3aa98_m.jpg
reading the images:flower_photo/roses\11102341464_508d558dfc_n.jpg
reading the images:flower_photo/roses\11233672494_d8bf0a3dbf_n.jpg
reading the images:flower_photo/roses\11694025703_9a906fedc1_n.jpg
reading the images:flower_photo/roses\118974357_0faa23cce9_n.jpg
reading the images:flower_photo/roses\11944957684_2cc806276e.jpg
reading the images:flower_photo/roses\12045735155_42547ce4e9_n.jpg
reading the images:flower_photo/roses\12165480946_c4a3fe182d_n.jpg
reading the images:flower_photo/roses\12202373204_34fb07205b.jpg
reading the images:flower_photo/roses\12238827553_cf427bfd51_n.jpg
reading the images:flower_photo/roses\12240165555_98625b1e88_n.jpg
reading the images:flower_photo/roses\12240303_80d87f77a3_n.jpg
reading th

reading the images:flower_photo/roses\16157873719_bf0bdf8558_n.jpg
reading the images:flower_photo/roses\16209331331_343c899d38.jpg
reading the images:flower_photo/roses\16229215579_e7dd808e9c.jpg
reading the images:flower_photo/roses\16258946661_f9739cdc0a.jpg
reading the images:flower_photo/roses\16316557109_7fc55c1cbc_m.jpg
reading the images:flower_photo/roses\16334786972_1b3e71cab8_m.jpg
reading the images:flower_photo/roses\16339359979_6d742660b8_n.jpg
reading the images:flower_photo/roses\16374919860_4e445de29f_n.jpg
reading the images:flower_photo/roses\16424992340_c1d9eb72b4.jpg
reading the images:flower_photo/roses\16449467833_d82aac5749_m.jpg
reading the images:flower_photo/roses\1645761726_2b1be95472.jpg
reading the images:flower_photo/roses\16476788181_0e2ffc719a.jpg
reading the images:flower_photo/roses\16484100863_979beacb08.jpg
reading the images:flower_photo/roses\16525204061_9b47be3726_m.jpg
reading the images:flower_photo/roses\16545641666_2781e542a0_m.jpg
reading th

reading the images:flower_photo/roses\2756028421_b3d5eea526_n.jpg
reading the images:flower_photo/roses\2777518561_105abc8cfc_n.jpg
reading the images:flower_photo/roses\2788276815_8f730bd942.jpg
reading the images:flower_photo/roses\2863863372_605e29c03e_m.jpg
reading the images:flower_photo/roses\2888138918_402096c7fb.jpg
reading the images:flower_photo/roses\2892056920_918c52889b_m.jpg
reading the images:flower_photo/roses\2949945463_366bc63079_n.jpg
reading the images:flower_photo/roses\2951375433_ae2726d9d2_m.jpg
reading the images:flower_photo/roses\295257304_de893fc94d.jpg
reading the images:flower_photo/roses\2960709681_e95940c0f0_n.jpg
reading the images:flower_photo/roses\2973256732_1926295f35.jpg
reading the images:flower_photo/roses\2976723295_b16ab04231.jpg
reading the images:flower_photo/roses\2980099495_cf272e90ca_m.jpg
reading the images:flower_photo/roses\298670754_f25edda891.jpg
reading the images:flower_photo/roses\3026375835_a20ecdd140_m.jpg
reading the images:flowe

...

reading the images:flower_photo/sunflowers\17433282043_441b0a07f4_n.jpg
reading the images:flower_photo/sunflowers\175638423_058c07afb9.jpg
reading the images:flower_photo/sunflowers\1788133737_b1133d1aa7.jpg
reading the images:flower_photo/sunflowers\18097401209_910a46fae1_n.jpg
reading the images:flower_photo/sunflowers\18237156988_9ceb46a8de_n.jpg
reading the images:flower_photo/sunflowers\18237215308_a158d49f28_n.jpg
reading the images:flower_photo/sunflowers\18250039435_7654bc11be_n.jpg
reading the images:flower_photo/sunflowers\184682095_46f8607278.jpg
reading the images:flower_photo/sunflowers\184682320_73ccf74710.jpg
reading the images:flower_photo/sunflowers\184682506_8a9b8c662d.jpg
reading the images:flower_photo/sunflowers\184682652_c927a49226_m.jpg
reading the images:flower_photo/sunflowers\184682920_97ae41ce60_m.jpg
reading the images:flower_photo/sunflowers\184683023_737fec5b18.jpg
reading the images:flower_photo/sunflowers\18766965343_9f42d4bedc_m.jpg
reading the images:

reading the images:flower_photo/sunflowers\2721638730_34a9b7a78b.jpg
reading the images:flower_photo/sunflowers\2723995667_31f32294b4.jpg
reading the images:flower_photo/sunflowers\2729206569_9dd2b5a3ed.jpg
reading the images:flower_photo/sunflowers\2733109082_1351f6738a_n.jpg
reading the images:flower_photo/sunflowers\27465811_9477c9d044.jpg
reading the images:flower_photo/sunflowers\27466794_57e4fe5656.jpg
reading the images:flower_photo/sunflowers\274846229_990e976683_n.jpg
reading the images:flower_photo/sunflowers\274848710_5185cf33b1_n.jpg
reading the images:flower_photo/sunflowers\2759796022_55bd47bfa2_n.jpg
reading the images:flower_photo/sunflowers\2767658405_1e2043f44c_n.jpg
reading the images:flower_photo/sunflowers\2767688889_b176b0c3fb.jpg
reading the images:flower_photo/sunflowers\2803725948_5fd1f2fc99_n.jpg
reading the images:flower_photo/sunflowers\2807106374_f422b5f00c.jpg
reading the images:flower_photo/sunflowers\2816256710_a2d3616fae.jpg
reading the images:flower_ph

reading the images:flower_photo/sunflowers\4821232343_7e0bcfbfdf_n.jpg
reading the images:flower_photo/sunflowers\4831577091_f56157a5d5_n.jpg
reading the images:flower_photo/sunflowers\4846786944_2832c5c8b8.jpg
reading the images:flower_photo/sunflowers\4847062576_bae870479c_n.jpg
reading the images:flower_photo/sunflowers\4848279231_c4960e28b2_n.jpg
reading the images:flower_photo/sunflowers\4868595281_1e58083785.jpg
reading the images:flower_photo/sunflowers\4869189730_f47c124cda_n.jpg
reading the images:flower_photo/sunflowers\4871455214_8b5fb87ab6_n.jpg
reading the images:flower_photo/sunflowers\4872284527_ff52128b97.jpg
reading the images:flower_photo/sunflowers\4872892690_52dc25b0b4.jpg
reading the images:flower_photo/sunflowers\4877195645_791c3a83b9_m.jpg
reading the images:flower_photo/sunflowers\4878447831_e904c60cf8_n.jpg
reading the images:flower_photo/sunflowers\4890268276_563f40a193.jpg
reading the images:flower_photo/sunflowers\4893660821_eb7f02bef3_n.jpg
reading the imag

reading the images:flower_photo/sunflowers\6042014768_b57f0bfc79_n.jpg
reading the images:flower_photo/sunflowers\6050020905_881295ac72_n.jpg
reading the images:flower_photo/sunflowers\6053739964_a1d9ab3ed1_n.jpg
reading the images:flower_photo/sunflowers\6061175433_95fdb12f32_n.jpg
reading the images:flower_photo/sunflowers\6061177447_d8ce96aee0.jpg
reading the images:flower_photo/sunflowers\6074427492_1b5bab7848_n.jpg
reading the images:flower_photo/sunflowers\6080086410_17a02dcfb8.jpg
reading the images:flower_photo/sunflowers\6104713425_8a3277e34a.jpg
reading the images:flower_photo/sunflowers\6112510436_9fe06e695a_n.jpg
reading the images:flower_photo/sunflowers\6116210027_61923f4b64.jpg
reading the images:flower_photo/sunflowers\6122711533_2c219f0392_n.jpg
reading the images:flower_photo/sunflowers\6125761554_4e72819ce4_m.jpg
reading the images:flower_photo/sunflowers\6133988570_9dc778e622_m.jpg
reading the images:flower_photo/sunflowers\6140661443_bb48344226.jpg
reading the imag

reading the images:flower_photo/sunflowers\9302733302_2cb92cf275.jpg
reading the images:flower_photo/sunflowers\9309473873_9d62b9082e.jpg
reading the images:flower_photo/sunflowers\9339697826_88c9c4dc50.jpg
reading the images:flower_photo/sunflowers\9359374034_21fb12d613_n.jpg
reading the images:flower_photo/sunflowers\9375675309_987d32f99e_n.jpg
reading the images:flower_photo/sunflowers\9381481549_5a5d503e42_n.jpg
reading the images:flower_photo/sunflowers\9384867134_83af458a19_n.jpg
reading the images:flower_photo/sunflowers\9399711558_7cb9547cd3_n.jpg
reading the images:flower_photo/sunflowers\9410186154_465642ed35.jpg
reading the images:flower_photo/sunflowers\9427945592_07a2676945_n.jpg
reading the images:flower_photo/sunflowers\9431890901_cd11bda584_n.jpg
reading the images:flower_photo/sunflowers\9431896325_23bf6e8761.jpg
reading the images:flower_photo/sunflowers\9432335346_e298e47713_n.jpg
reading the images:flower_photo/sunflowers\9445830851_e9a126fd1d_n.jpg
reading the imag

In [9]:
from skimage import io,transform
import tensorflow as tf
import numpy as np
import requests


path1 = "testcase/rose.jpg"
#path to your test image
path2 = "testcase/sunflower.jpg"
#path3 = "E:/data/datasets/flower_photos/daisy/5547758_eea9edfd54_n.jpg"

flower_dict = {0:'roses',1:'sunflowers'}

w=100
h=100
c=3

def read_one_image(path):
    img = io.imread(path)
    img = transform.resize(img,(w,h))
    return np.asarray(img)

with tf.Session() as sess:
    data = []
    data1 = read_one_image(path1)
    data2 = read_one_image(path2)
    #data3 = read_one_image(path3)
    #data4 = read_one_image(path4)
    #data5 = read_one_image(path5)
    data.append(data1)
    data.append(data2)
    #data.append(data3)
    #data.append(data4)
    #data.append(data5)
    #the path to your trained model
    saver = tf.train.import_meta_graph('data/model.ckpt.meta')
    saver.restore(sess,tf.train.latest_checkpoint('data/'))
    #save your check point
    graph = tf.get_default_graph()
    x = graph.get_tensor_by_name("x:0")
    feed_dict = {x:data}

    logits = graph.get_tensor_by_name("logits_eval:0")

    classification_result = sess.run(logits,feed_dict)

    #print the predicted matrix
    print(classification_result)
    #print max value in rows
    print(tf.argmax(classification_result,1).eval())
    #get flower name in dict
    output = []
    output = tf.argmax(classification_result,1).eval()
    for i in range(len(output)):
        print("The sample you just input is "+flower_dict[output[i]])

C:\Users\lyl5380\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


INFO:tensorflow:Restoring parameters from data/model.ckpt
[[ 46.527374  36.864464 -14.007546 -80.253105 -17.600328]
 [ 22.110088  35.13613  -10.905572 -57.943962 -13.165587]]
[0 1]
The sample you just input is roses
The sample you just input is sunflowers
